In [ ]:
# cyber_marketing_visualization.py
# Connects to MySQL (cyber_marketing DB), runs stored procedures
# from campaigns_table.sql, influencers_table.sql, and regions_table.sql,
# then visualizes the results using matplotlib + seaborn.

import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mysql.connector import Error

sns.set(style="whitegrid", context="notebook")
plt.rcParams["figure.figsize"] = (10, 5)

# ---------------------------------------------------------
# ✅ Database Configuration (Your Credentials)
# ---------------------------------------------------------
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "Bala@0505",      # Your given password
    "database": "cyber_marketing" # Your database name
}

# ---------------------------------------------------------
# Function: Run Stored Procedure
# ---------------------------------------------------------
def run_stored_procedure(proc_name, params=None):
    """
    Executes a stored procedure and returns the result as a DataFrame.
    """
    try:
        connection = mysql.connector.connect(**db_config)
        cursor = connection.cursor()

        print(f"\n🔹 Executing stored procedure: {proc_name}")
        if params:
            cursor.callproc(proc_name, params)
        else:
            cursor.callproc(proc_name)

        df = None
        for result in cursor.stored_results():
            rows = result.fetchall()
            cols = [col[0] for col in result.description]
            df = pd.DataFrame(rows, columns=cols)
        return df

    except Error as e:
        print(f"⚠️ Error executing {proc_name}: {e}")
        return None
    finally:
        if cursor:
            cursor.close()
        if connection.is_connected():
            connection.close()

# ---------------------------------------------------------
# Visualization Functions for Campaigns
# ---------------------------------------------------------
def viz_campaign_types_by_roi(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(8,4))
        sns.barplot(data=df, x='campaign_type', y='avg_roi')
        plt.title("Average ROI by Campaign Type")
        plt.xlabel("Campaign Type"); plt.ylabel("Avg ROI (%)")
        plt.xticks(rotation=20); plt.tight_layout(); plt.show()

def viz_budget_bins_roi(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(8,4))
        sns.barplot(data=df, x='budget_range', y='avg_roi')
        plt.title("Average ROI by Budget Range")
        plt.xlabel("Budget Range"); plt.ylabel("Avg ROI (%)")
        plt.tight_layout(); plt.show()

def viz_top_campaigns_by_roi(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(10,4))
        sns.barplot(data=df.sort_values("roi", ascending=False),
                    x=df["campaign_id"].astype(str), y='roi')
        plt.title("Top Campaigns by ROI")
        plt.xlabel("Campaign ID"); plt.ylabel("ROI (%)")
        plt.xticks(rotation=45); plt.tight_layout(); plt.show()

def viz_top_campaigns_engagement_ctr(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(10,4))
        sns.barplot(data=df, x=df["campaign_id"].astype(str), y='engagement_click_through_rate_score')
        plt.title("Top Campaigns by Engagement + CTR Score")
        plt.xlabel("Campaign ID"); plt.ylabel("Engagement+CTR")
        plt.xticks(rotation=45); plt.tight_layout(); plt.show()

def viz_worst_campaigns_by_roi(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(10,4))
        sns.barplot(data=df.sort_values("roi"), x=df["campaign_id"].astype(str), y='roi')
        plt.title("Worst Campaigns by ROI")
        plt.xlabel("Campaign ID"); plt.ylabel("ROI (%)")
        plt.xticks(rotation=45); plt.tight_layout(); plt.show()

# ---------------------------------------------------------
# Visualization Functions for Influencers
# ---------------------------------------------------------
def viz_top_platforms_by_influencer_count(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(8,4))
        sns.barplot(data=df, x='platform', y='influencer_count')
        plt.title("Influencer Count by Platform")
        plt.xlabel("Platform"); plt.ylabel("Count")
        plt.tight_layout(); plt.show()

def viz_top_influencers_by_campaign_count(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(10,4))
        sns.barplot(data=df.sort_values("campaign_type", ascending=False),
                    x='influencer_name', y='campaign_type')
        plt.title("Top Influencers by Campaign Count")
        plt.xlabel("Influencer"); plt.ylabel("Campaign Count")
        plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()

def viz_top_influencers_by_avg_roi(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(10,4))
        sns.barplot(data=df, x='influencer_name', y='avg_roi')
        plt.title("Top Influencers by Average ROI")
        plt.xlabel("Influencer"); plt.ylabel("Avg ROI (%)")
        plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()

def viz_platform_performance_by_roi(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(8,4))
        sns.barplot(data=df, x='platform', y='avg_roi')
        plt.title("Platform Performance by Avg ROI")
        plt.xlabel("Platform"); plt.ylabel("Avg ROI (%)")
        plt.tight_layout(); plt.show()

def viz_top_influencers_by_engagement(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(10,4))
        sns.barplot(data=df, x='influencer_name', y='avg_engagement_rate')
        plt.title("Top Influencers by Avg Engagement Rate")
        plt.xlabel("Influencer"); plt.ylabel("Avg Engagement Rate")
        plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()

# ---------------------------------------------------------
# Visualization Functions for Regions
# ---------------------------------------------------------
def viz_top_regions_by_influencer_count(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(8,4))
        sns.barplot(data=df, x='region_name', y='influencer_count')
        plt.title("Influencer Count by Region")
        plt.xlabel("Region"); plt.ylabel("Count")
        plt.tight_layout(); plt.show()

def viz_top_regions_by_total_roi(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(8,4))
        sns.barplot(data=df, x='region_name', y='total_roi')
        plt.title("Total ROI by Region")
        plt.xlabel("Region"); plt.ylabel("Total ROI")
        plt.tight_layout(); plt.show()

def viz_top_regions_impressions_reach(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(10,5))
        melted = df.melt(id_vars='region_name',
                         value_vars=['total_impressions','total_reach'],
                         var_name='metric', value_name='value')
        sns.barplot(data=melted, x='region_name', y='value', hue='metric')
        plt.title("Total Impressions and Reach by Region")
        plt.xlabel("Region"); plt.ylabel("Counts")
        plt.tight_layout(); plt.show()

def viz_top_regions_by_avg_engagement(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(8,4))
        sns.barplot(data=df, x='region_name', y='avg_engagement_rate')
        plt.title("Average Engagement Rate by Region")
        plt.xlabel("Region"); plt.ylabel("Avg Engagement Rate")
        plt.tight_layout(); plt.show()

def viz_top_regions_by_conversions(df):
    if df is not None and not df.empty:
        plt.figure(figsize=(8,4))
        sns.barplot(data=df, x='region_name', y='total_conversions')
        plt.title("Total Conversions by Region")
        plt.xlabel("Region"); plt.ylabel("Conversions")
        plt.tight_layout(); plt.show()

# ---------------------------------------------------------
# Main Menu to Choose & Run Stored Procedures
# ---------------------------------------------------------
def main():
    menu = [
        ("Campaigns: Top campaign types by ROI", "GetTopCampaignTypesByROI", viz_campaign_types_by_roi, (5,)),
        ("Campaigns: Budget bins ROI", "GetBudgetBinsROI", viz_budget_bins_roi, (5,)),
        ("Campaigns: Top campaigns by ROI", "GetTopCampaignsByROI", viz_top_campaigns_by_roi, (5,)),
        ("Campaigns: Top campaigns by Engagement+CTR", "GetTopCampaignsByEngagementCTR", viz_top_campaigns_engagement_ctr, (5,)),
        ("Campaigns: Worst campaigns by ROI", "GetWorstCampaignsByROI", viz_worst_campaigns_by_roi, (5,)),
        ("Influencers: Top platforms by influencer count", "GetTopPlatformsByInfluencerCount", viz_top_platforms_by_influencer_count, (5,)),
        ("Influencers: Top influencers by campaign count", "GetTopInfluencersByCampaignCount", viz_top_influencers_by_campaign_count, (5,)),
        ("Influencers: Top influencers by avg ROI", "GetTopInfluencersByAvgROI", viz_top_influencers_by_avg_roi, (5,)),
        ("Influencers: Platform performance by ROI", "GetPlatformPerformanceByROI", viz_platform_performance_by_roi, (5,)),
        ("Influencers: Top influencers by engagement", "GetTopInfluencersByEngagement", viz_top_influencers_by_engagement, (5,)),
        ("Regions: Top regions by influencer count", "GetTopRegionsByInfluencerCount", viz_top_regions_by_influencer_count, (5,)),
        ("Regions: Top regions by total ROI", "GetTopRegionsByTotalROI", viz_top_regions_by_total_roi, (5,)),
        ("Regions: Top regions by impressions & reach", "GetTopRegionsByImpressionsReach", viz_top_regions_impressions_reach, (5,)),
        ("Regions: Top regions by avg engagement", "GetTopRegionsByAvgEngagement", viz_top_regions_by_avg_engagement, (5,)),
        ("Regions: Top regions by conversions", "GetTopRegionsByConversions", viz_top_regions_by_conversions, (5,))
    ]

    print("\n================ CYBER MARKETING DASHBOARD ================")
    for i, item in enumerate(menu, start=1):
        print(f"{i:2d}. {item[0]}")
    print(" 0. Exit")
    print("==========================================================")

    try:
        choice = int(input("Enter choice number: ").strip())
    except ValueError:
        print("Please enter a valid number.")
        return

    if choice == 0:
        print("Exiting program.")
        return

    if 1 <= choice <= len(menu):
        title, proc, func, params = menu[choice - 1]
        print(f"\n➡ Running: {title}")
        df = run_stored_procedure(proc, params)
        if df is not None and not df.empty:
            print("\nData Preview:\n", df.head())
            func(df)
        else:
            print("No data returned from procedure.")
    else:
        print("Invalid choice number.")

if __name__ == "__main__":
    main()



================ CYBER MARKETING DASHBOARD ================
 1. Campaigns: Top campaign types by ROI
 2. Campaigns: Budget bins ROI
 3. Campaigns: Top campaigns by ROI
 4. Campaigns: Top campaigns by Engagement+CTR
 5. Campaigns: Worst campaigns by ROI
 6. Influencers: Top platforms by influencer count
 7. Influencers: Top influencers by campaign count
 8. Influencers: Top influencers by avg ROI
 9. Influencers: Platform performance by ROI
10. Influencers: Top influencers by engagement
11. Regions: Top regions by influencer count
12. Regions: Top regions by total ROI
13. Regions: Top regions by impressions & reach
14. Regions: Top regions by avg engagement
15. Regions: Top regions by conversions
 0. Exit
